# ESS refactoring
Builds on top of xarray-einstats.

In [1]:
import xarray as xr
import arviz as az
import numpy as np
from scipy.fftpack import next_fast_len

from utils import autocov, autocorr, _ess_mean, _ess_bulk, ess_einstats

In [2]:
ds = az.load_arviz_data("centered_eight").posterior
ds

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 500, school: 8)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 492 493 494 495 496 497 498 499
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       (chain, draw) float64 ...
    theta    (chain, draw, school) float64 ...
    tau      (chain, draw) float64 ...
Attributes:
    created_at:                 2019-06-21T17:36:34.398087
    inference_library:          pymc3
    inference_library_version:  3.7

In [3]:
xe_cov = autocov(ds.theta)
az_cov = az.autocov(ds.theta.values, axis=1)
np.allclose(xe_cov.transpose(*ds.theta.dims), az_cov)

True

In [4]:
xe_corr = autocorr(ds.theta)
az_corr = az.autocorr(ds.theta.values, axis=1)
np.allclose(xe_corr.transpose(*ds.theta.dims), az_corr)

True

In [5]:
ds = az.load_arviz_data("centered_eight").posterior
ds_dask = ds.chunk(chunks=dict(school=2))

In [6]:
ess_einstats(ds_dask, method="mean", dask="allowed").compute()

HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 494 in H5O__attr_open_by_name(): can't locate attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Object not found
HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 1:
  #000: H5A.c line 528 in H5Ao

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       float64 246.4
    theta    (school) float64 394.2 567.1 503.8 613.3 393.0 554.8 527.1 573.6
    tau      float64 142.3

In [7]:
ess_einstats(ds, method="mean")

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       float64 246.4
    theta    (school) float64 394.2 567.1 503.8 613.3 393.0 554.8 527.1 573.6
    tau      float64 142.3

In [8]:
az.ess(ds, method="mean")

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       float64 246.4
    theta    (school) float64 394.2 567.1 503.8 613.3 393.0 554.8 527.1 573.6
    tau      float64 142.3

In [9]:
ess_einstats(ds_dask, method="bulk", dask="allowed").compute()

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       float64 250.3
    theta    (school) float64 347.8 471.4 463.2 502.7 380.3 516.2 402.0 449.4
    tau      float64 78.97

In [10]:
ess_einstats(ds, method="bulk")

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       float64 250.3
    theta    (school) float64 347.8 471.4 463.2 502.7 380.3 516.2 402.0 449.4
    tau      float64 78.97

In [11]:
az.ess(ds, method="bulk")

<xarray.Dataset>
Dimensions:  (school: 8)
Coordinates:
  * school   (school) object 'Choate' 'Deerfield' ... "St. Paul's" 'Mt. Hermon'
Data variables:
    mu       float64 250.3
    theta    (school) float64 347.8 471.4 463.2 502.7 380.3 516.2 402.0 449.4
    tau      float64 78.97